In [1]:
import faker
import random
from datetime import datetime
import numpy as np

In [2]:
fake = faker.Faker()

In [3]:
num_users = int(1e+6)
num_groups = int(1e+4)
num_reservoirs = int(2e+5)
num_measurements = int(2e+6)

min_users_in_group = 2
max_users_in_group = 200

In [4]:
import psycopg2
from psycopg2 import Error
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT

connection = psycopg2.connect(dbname="db-in-es",
                            user="db-in-es",
                            password="db-in-es",
                            host="127.0.0.1",
                            port="5432")
connection.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)
cur = connection.cursor()

In [ ]:
# user + token

sys_user = []
token = []

arr_token_user_ref = list(range(1, num_users + 1))
random.shuffle(arr_token_user_ref)

for i in range(num_users):
#     if i%10000 == 0: 
#         print(i)
    login = fake.user_name() + str(i)
    password = fake.md5()
    description = fake.text(256)
    first_name = fake.first_name()
    second_name = fake.first_name()
    last_name = fake.last_name()
    phone = fake.phone_number() + str(i)
    email = fake.email() + str(i)
    is_online= random.choice(['true', 'false'])
    
    sys_user.append(
        {
            'login': login,
            'password': password,
            'description': description,
            'first_name': first_name,
            'second_name': second_name,
            'last_name': last_name,
            'phone': phone,
            'email': email,
            'is_online': is_online,
        }
    )

    content = bytes(fake.text(128), 'utf-8')
    sys_user_ref = arr_token_user_ref[i]

    token.append(
        {
            'content': content,
            'sys_user_ref': sys_user_ref,
        }
    )

In [6]:
for u in sys_user:
    try:
        cur.execute('''
        INSERT INTO sys_user 
        (login,password,description,first_name,second_name,last_name,phone,email,is_online) 
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)''',
        (u['login'],u['password'],u['description'], 
        u['first_name'],u['second_name'],u['last_name'],
        u['phone'],u['email'],u['is_online']))
    except (Exception, Error) as error:
        print(error)

In [7]:
for t in token:
    try:
        cur.execute('''INSERT INTO token (sys_user_ref,content) VALUES (%s, %s)''',
        (t['sys_user_ref'],t['content']))
    except (Exception, Error) as error:
        print(error)

In [8]:
del sys_user
del token

In [ ]:
# groups, reservoirs

sys_group = []

for i in range(num_groups):
#     if i%10000 == 0: 
#         print(i)
    description = fake.text(256)
    name = fake.company() + str(i)
    sys_group.append(
        {
            'description': description,
            'name': name,
        }
    )

reservoir = []

for i in range(num_reservoirs):
#     if i%10000 == 0: 
#         print(i)
    description = fake.text(256)
    name = fake.city() + str(i)
    reservoir.append(
        {
            'description': description,
            'name': name,
        }
    )

In [11]:
try:
    for g in sys_group:
        cur.execute('''INSERT INTO sys_group (description,name) VALUES (%s, %s)''',
        (g['description'],g['name']))
        
except (Exception, Error) as error:
    print(error)

In [12]:
try:
    for r in reservoir:
        cur.execute('''INSERT INTO reservoir (description,name) VALUES (%s, %s)''',
        (r['description'],r['name']))
        
except (Exception, Error) as error:
    print(error)

In [13]:
del sys_group
del reservoir

In [ ]:
# link_sys_user_sys_group

link_sys_user_sys_group = []
user_ids = list(range(1, num_users + 1))

for gid in list(range(1, num_groups + 1)):

    tmp = np.random.choice(user_ids, random.randint(min_users_in_group, max_users_in_group))
    is_primary = random.choice(['true', 'false'])
    is_admin = random.choice(['true', 'false'])
    
    for uid in tmp:
        link_sys_user_sys_group.append(
        {
            'sys_user_ref': int(uid),
            'sys_group_ref': int(gid),
            'is_primary': is_primary,
            'is_admin': is_admin,
        }
    )



In [20]:
len(link_sys_user_sys_group)

1001014

In [21]:
try:
    for l in link_sys_user_sys_group:
        cur.execute('''INSERT INTO link_sys_user_sys_group 
        (sys_user_ref,sys_group_ref,is_primary,is_admin) 
        VALUES (%s, %s, %s, %s)''',
        (l['sys_user_ref'],l['sys_group_ref'],l['is_primary'],l['is_admin']))
        
except (Exception, Error) as error:
    print(error)

In [ ]:
# measurement

measurement = []

reservoir_ids = list(range(1, num_reservoirs + 1))

for i in range(num_measurements):
    lat = fake.latitude()
    lon = fake.longitude()
    date = datetime.now()
    ph = random.uniform(0, 14)
    hardness = random.uniform(40, 300)       
    solids = random.uniform(300, 50000)          
    chloramines = random.uniform(0, 14)  
    sulfate = random.uniform(100, 500)          
    conductivity = random.uniform(180, 1000)
    organic_carbon = random.uniform(2, 30)       
    trihalomethanes = random.uniform(0, 140)    
    turbidity = random.uniform(1, 7)
    
    tmp = random.choice(link_sys_user_sys_group)
    sys_user_ref = int(tmp['sys_user_ref'])
    sys_group_ref = int(tmp['sys_group_ref'])
    reservoir_ref = random.choice(reservoir_ids)
    
    measurement.append(
        {
            'lon': lon,
            'lat': lat,
            'date': date,
            'ph': ph,                
            'hardness': hardness,          
            'solids': solids,             
            'chloramines': chloramines,        
            'sulfate': sulfate,             
            'conductivity': conductivity,         
            'organic_carbon': organic_carbon,        
            'trihalomethanes': trihalomethanes,      
            'turbidity': turbidity,
            'sys_user_ref': sys_user_ref,
            'sys_group_ref': sys_group_ref,
            'reservoir_ref': reservoir_ref,
        }
    )


In [24]:
try:        
    for m in measurement:
        cur.execute('''
        INSERT INTO measurement
        (lon,lat,date,
        sys_user_ref,sys_group_ref,reservoir_ref,
        ph,hardness,solids,chloramines,sulfate,conductivity,
        organic_carbon,trihalomethanes,turbidity) 
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s,%s, %s, %s, %s, %s, %s)''',
        (m['lon'],m['lat'],m['date'],
        m['sys_user_ref'],m['sys_group_ref'],m['reservoir_ref'],
        m['ph'],m['hardness'],m['solids'],m['chloramines'],m['sulfate'],m['conductivity'],
        m['organic_carbon'],m['trihalomethanes'],m['turbidity']))
        
except (Exception, Error) as error:
    print(error)

In [25]:
del link_sys_user_sys_group
del measurement

In [27]:
cur.close()
connection.close()